In [1]:
%load_ext autoreload
%autoreload 2
from IPython.display import Audio

In [22]:
from datasets.audio_compress import LibriTTS
from datasets.utils.diskcache import DiskCachedDataset


In [31]:
data_not_cache = LibriTTS('/home/romain/datasets/LibriTTS/', sampling_freq=16_000, sample_length=512, max_sample=1_000)

directory already exist
Wave files are resampled to 16000Hz
Chunk map loaded from /home/romain/datasets/LibriTTS/debug/16000/full/512_map.pkl
Waves files are splited into 512 samples length


In [45]:
data_not_cache

AttributeError: 'LibriTTS' object has no attribute '__iter__'

In [32]:
len(data_not_cache)

1000

In [33]:
data_cache = DiskCachedDataset(data_not_cache, cache_path='/home/romain/datasets/cache/LibriTTS')


1000


In [34]:
# create cache 
for i in range(data_cache.__len__()):
    data_cache[i]

In [36]:
import timeit
import random

# Assume `dataset` is your PyTorch Dataset
# Set the number of samples for timing
num_samples = 1000

# Generate a list of random indices
indices = [random.randint(0, len(data_not_cache) - 1) for _ in range(num_samples)]

# Time the average access time
average_time = timeit.timeit(
    stmt="data_not_cache[random.choice(indices)]",
    setup="from __main__ import data_not_cache, indices, random",
    number=num_samples
) / num_samples
average_time 
# around 5ms 
# it's too slow considering linear increase by the batch size,
# charging a 256 batch it will take, 256 * 5ms = 1.280 s
# we need to consider caching

0.005280411747000016

In [37]:
# Assume `dataset` is your PyTorch Dataset
# Set the number of samples for timing
num_samples = 1000

# Generate a list of random indices
indices = [random.randint(0, len(data_cache) - 1) for _ in range(num_samples)]

# Time the average access time
average_time = timeit.timeit(
    stmt="data_cache[random.choice(indices)]",
    setup="from __main__ import data_cache, indices, random",
    number=num_samples
) / num_samples
average_time # around 5ms 
# it's too slow considering linear increase by the batch size,
# charging a 256 batch it will take, 256 * 5ms = 1.280 s
# we need to consider caching

0.00015699946199993064

In [38]:
from torch.utils.data import DataLoader
from datasets.utils.pad_tensors import PadTensors
dataset = DataLoader(data_cache, batch_size=100, collate_fn=PadTensors())


In [39]:
num_samples = 10
it = iter(dataset)
average_time = timeit.timeit(
    stmt="next(it)",
    setup="from __main__ import it",
    number=num_samples
) / num_samples
average_time

0.01626775599997927

In [41]:
for d in dataset:
    pass

In [439]:

import torch
import torch.nn.functional as F
torch.backends.cudnn.deterministic = True
def ema_parallel(x:torch.tensor, alpha):
    """
    Compute the exponential moving average (EMA) in parallel using PyTorch.
    
    Args:
        x (torch.Tensor): Input sequence of shape (N, T, C) where N is the batch size and T is the sequence length.
        alpha (float): Smoothing factor, 0 < alpha < 1.
        
    Returns:
        torch.Tensor: EMA of shape (N, T, C).
    """
    # put the temporal dimension at the end
    x = torch.permute(x, (0,2,1))
    # Compute weights for the convolution
    i = torch.arange(x.shape[-1], device=x.device, dtype=torch.int,)  # Indices
    weights = (1 - alpha) * (alpha**i)  # Compute weights as a tensor
    weights = weights.flip(0).broadcast_to(x.shape[-2], 1, x.shape[-1])    # Pad the input to simulate u_0 = 0
    x_padded = F.pad(x, (weights.shape[-1] - 1, 0), mode="constant", value=0.0)  # Pad on the left
    ema = F.conv1d(x_padded, weights, groups=x.shape[-2])
    ema = torch.permute(ema, (0,2,1))
    return ema

In [440]:
def ema_sequential(x:torch.Tensor, alpha: float):
    u = torch.zeros_like(x[:, 0])
    buffer = []
    for x_t in x.unbind(1):
        u = alpha * u + (1 - alpha) * x_t
        buffer.append(u)
    return torch.stack(buffer, 1)

In [441]:
x = torch.arange(1.0, 101.0, step=1.0, requires_grad=True).reshape(1, 10, 10)
alpha = torch.tensor(0.9, dtype=torch.float, requires_grad=True)

In [449]:
ema_p = ema_parallel(x, alpha).mean()
df_dx, df_dy = torch.autograd.grad(ema_p, [x, alpha])


In [450]:
df_dx
df_dy

tensor(-127.8128)

In [448]:
ema_seq = ema_sequential(x, 0.9)

In [409]:
ema_seq.shape


torch.Size([1, 10, 10])

In [410]:
ema_p[0,:, 0]

tensor([ 0.1000,  1.1900,  3.1710,  5.9539,  9.4585, 13.6127, 18.3514, 23.6163,
        29.3546, 35.5192])

In [411]:
ema_seq[0,:, 0]

tensor([ 0.1000,  1.1900,  3.1710,  5.9539,  9.4585, 13.6127, 18.3514, 23.6163,
        29.3546, 35.5192])